In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch import optim

In [2]:
import sys
sys.path.insert(1, "../../../")

In [3]:
from train import *
from data_preprocessing import *
from Models.yolov8cls_path import Model

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
model = Model(num_classes=10, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [6]:
data_path = '../../../../datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [7]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [8]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [9]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations, half=False, show=False)

In [10]:
epochs=10
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [12]:
loss_fn = nn.NLLLoss()

In [13]:
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [14]:
lr = 0.01
for i in range(10):
    print(lr)
    lr = lr * 0.9

0.01
0.009000000000000001
0.008100000000000001
0.007290000000000001
0.006561000000000002
0.005904900000000002
0.005314410000000002
0.004782969000000002
0.004304672100000002
0.003874204890000002



Profiling your personal module 
https://pytorch.org/tutorials/beginner/profiler.html

In [15]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../../log/YOLOv8cls-version-5/training/')

2024-12-05 12:46:37.436304 Epoch 1 
2024-12-05 12:46:52.693472 Batch 1 
2024-12-05 12:46:53.831903 Batch 2 
2024-12-05 12:46:53.930421 Batch 3 
2024-12-05 12:46:54.039948 Batch 4 
2024-12-05 12:46:54.151486 Batch 5 
2024-12-05 12:46:54.260866 Batch 6 
2024-12-05 12:46:54.360668 Batch 7 
2024-12-05 12:46:54.466190 Batch 8 
2024-12-05 12:46:54.744246 Batch 9 
2024-12-05 12:46:55.151375 Batch 10 
2024-12-05 12:46:55.250909 Batch 11 
2024-12-05 12:46:55.350437 Batch 12 
2024-12-05 12:46:55.692822 Batch 13 
2024-12-05 12:46:56.285222 Batch 14 
2024-12-05 12:46:56.383754 Batch 15 
2024-12-05 12:46:56.483277 Batch 16 
2024-12-05 12:46:56.733310 Batch 17 
2024-12-05 12:46:57.378648 Batch 18 
2024-12-05 12:46:57.482169 Batch 19 
2024-12-05 12:46:57.580686 Batch 20 
2024-12-05 12:46:57.692308 Batch 21 
2024-12-05 12:46:58.388361 Batch 22 
2024-12-05 12:46:58.491487 Batch 23 
2024-12-05 12:46:58.593017 Batch 24 
2024-12-05 12:46:58.706542 Batch 25 
2024-12-05 12:46:59.427340 Batch 26 
2024-12-05 

NameError: name 'profiler' is not defined

In [16]:
print(prof.key_averages().table(row_limit=-1))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         0.55%     766.350ms       100.00%      138.347s       17.293s      19.45 Mb    -466.33 Mb             8            --  
                                          Training Loop         0.03%      38.355ms        51.31%       70.990s        8.874s      19.43 Mb    -137.83 Mb             8            --  
                     Validation Loop Before Data Loader         4.35%        6.0

In [17]:
save_train_outputs('../../log/YOLOv8cls-version-5/training', history, gradient_stats)